In [3]:
import os
import sys


ModuleNotFoundError: No module named 'pyspark'

In [4]:
os.environ["PYSPARK_PYTHON"]="C:\Spark\spark-3.3.2-bin-hadoop2\python"
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.3.2-bin-hadoop2"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.9.5-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip") 

In [5]:
# initialising the session with 14 GB driver memory
MAX_MEMORY = "14G"
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("demo") \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()    
    

spark

In [6]:
spark.sparkContext.getConf().get('spark.driver.memory')

'14G'

In [7]:
data = spark.read.option("header","true").csv("../data/r_hosp_dataset.csv")
data.printSchema()

root
 |-- subject_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dod: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- so2: string (nullable = true)
 |-- po2: string (nullable = true)
 |-- pco2: string (nullable = true)
 |-- fio2: string (nullable = true)
 |-- aado2: string (nullable = true)
 |-- ph: string (nullable = true)
 |-- baseexcess: string (nullable = true)
 |-- bg2_bicarbonate: string (nullable = true)
 |-- totalco2: string (nullable = true)
 |-- bg2_hematocrit: string (nullable = true)
 |-- bg2_haemoglobin: string (nullable = true)
 |-- carboxyhemoglobin: string (nullable = true)
 |-- methemoglobin: string (nullable = true)
 |-- bg2_chloride: string (nullable = true)
 |-- bg2_calcium: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- bg2_potassium: string (nullable = true)
 |-- bg2_sodium: string (nullable = true)
 |-- bg2_lactate: string (nullable = true)
 |-- bg2_glucose:

In [8]:
data.count()

431231

In [9]:
data.describe().toPandas()

,summary,subject_id,gender,dod,hadm_id,age,so2,po2,pco2,fio2,...,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,crp,charlson_comorbidity_index
0,count,431231,431231,106218,431231,431231,27945,62060,62050,16541,...,21768,148620,13409,13897,74281,65427,3717,92776,20994,431231
1,mean,1.5007664969607936E7,None,None,2.5003138264725868E7,59.26944843426296,92.06972982644479,197.93195294875926,48.15066881547139,64.99111299195937,...,109.67851892686512,1.4621228636791646,3.0113729584607327,1.7329351658631367,727.7982525814139,11.967495070842311,250.6031746031746,389.837005259981,66.98933552443553,3.5870218050186558
2,stddev,2877497.8062713076,None,None,2888180.402945388,19.207987665245223,12.191322751514052,136.2990225302963,14.732684122524283,24.739972885601127,...,216.2017369997382,3.557465027266691,5.138052210796493,2.325996291676031,6729.51889196154,38.24575537849562,386.639868567945,1562.3742701386582,74.66665896488824,3.031824233136227
3,min,10000032,F,2110-01-25,20000019,100.0098843901973060,100,0,10,100,...,10,0.1,0.1,0.1,10,1,10,100,0.1,0
4,max,19999987,M,2212-01-22,29999928,99.9662204176348223,99.9,99,99,99,...,996,9.9,9.9,9.9,999,99,999,9990,99.9,9


- All the rows has subject, gender, age, admission id (hadm_id) and charlson_comorbidity_index.


To Do
- Check for valid gender values and remove invalid genders (those other than 'M' and 'F').
- Check for valid age values.
- impute missing values by calculating the average of the value between the last & next reading for the same subject.
- Remove columns with more than 50% missing values after imputation.
- remove rows which has data in less than 25% of the columns.

#### Check for valid gender values and remove invalid genders (those other than 'M' and 'F').

In [10]:
#checking for gender values
data.groupby("gender").count().show()

+------+------+
|gender| count|
+------+------+
|     F|224990|
|     M|206241|
+------+------+



Gender column values looks good.

#### type conversion for the columns data

Except for Gender, all other columns contain decimal values. Hence converting every column type to decimal with 2 decimal values.

In [11]:
# converting all number columns to double of precision 2 except for gender
import pyspark.sql.functions as F

cols = data.columns
cols.remove("gender")
data = data.select(*(F.round(F.col(c).cast("double"), 2).alias(c) for c in cols), "gender")
print(data.columns)
print(data.printSchema())
data.describe().toPandas()

['subject_id', 'dod', 'hadm_id', 'age', 'so2', 'po2', 'pco2', 'fio2', 'aado2', 'ph', 'baseexcess', 'bg2_bicarbonate', 'totalco2', 'bg2_hematocrit', 'bg2_haemoglobin', 'carboxyhemoglobin', 'methemoglobin', 'bg2_chloride', 'bg2_calcium', 'temperature', 'bg2_potassium', 'bg2_sodium', 'bg2_lactate', 'bg2_glucose', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'atypical_lymphocytes', 'bands', 'immature_granulocytes', 'metamyelocytes', 'nrbc', 'troponin_t', 'cm_ck_mb', 'ntprobnp', 'albumin', 'globulin', 'total_protein', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'd_dimer', 'fibrinogen', 'thrombin', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'rdwsd', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'amylase', 'bilirubin_total', 'bi

,summary,subject_id,dod,hadm_id,age,so2,po2,pco2,fio2,aado2,...,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,crp,charlson_comorbidity_index,gender
0,count,431231,0,431231,431231,27945,62060,62050,16541,5437,...,148620,13409,13897,74281,65427,3717,92776,20994,431231,431231
1,mean,1.5007664969607936E7,None,2.5003138264725868E7,59.2694490655819,92.06972982644479,197.93195294875926,48.15066881547139,64.99111299195937,492.72652565753174,...,1.4621228636791646,3.0113729584607327,1.7329351658631367,727.7982525814139,11.967495070842311,250.6031746031746,389.837005259981,66.98933552443553,3.5870218050186558,None
2,stddev,2877497.8062713076,None,2888180.402945388,19.20798714444107,12.191322751514052,136.2990225302963,14.732684122524283,24.739972885601127,122.97508532137482,...,3.557465027266691,5.138052210796493,2.325996291676031,6729.51889196154,38.24575537849562,386.639868567945,1562.3742701386582,74.66665896488824,3.031824233136227,None
3,min,1.0000032E7,None,2.0000019E7,18.0,7.0,0.0,10.0,21.0,0.12,...,0.1,0.1,0.1,4.0,1.0,3.0,31.0,0.1,0.0,F
4,max,1.9999987E7,None,2.9999928E7,103.17,100.0,4242.0,246.0,100.0,743.0,...,87.2,68.0,29.6,591950.0,673.0,7380.0,377000.0,608.1,20.0,M


- The min and max age values looks good.

#### Impute missing values - Calculating the average of the values between the prev & next admission reading for the same subject.

In [12]:
# sample for verification
data.filter((data.subject_id == 10040025)).select("subject_id","age","basophils_abs", "db_wbc", "platelet", "ast").orderBy("age").show()

+-----------+-----+-------------+------+--------+----+
| subject_id|  age|basophils_abs|db_wbc|platelet| ast|
+-----------+-----+-------------+------+--------+----+
|1.0040025E7|64.21|         null|  null|   202.0|null|
|1.0040025E7|66.51|         null|  null|    null|null|
|1.0040025E7|66.56|         null|   8.9|   275.0|null|
|1.0040025E7| 66.8|         null|   6.2|   258.0|null|
|1.0040025E7|68.46|         null|  12.6|   281.0|null|
|1.0040025E7|68.59|         0.01|  11.3|   307.0|null|
|1.0040025E7|68.86|         null|   9.8|   275.0|null|
|1.0040025E7|68.93|         null|   8.6|   323.0|null|
|1.0040025E7|68.99|         null|  12.6|   366.0|null|
|1.0040025E7|69.06|         0.11|  16.7|   349.0|16.0|
+-----------+-----+-------------+------+--------+----+



In [ ]:
# Before impute
data.toPandas().to_csv("../data/EDA/before_imputation_all.csv")

In [13]:
# Use window function to impute missing values.
# https://sqlrelease.com/get-the-first-non-null-value-per-group-spark-dataframe
from pyspark.sql.window import Window
subject_win_prev = Window.partitionBy("subject_id").orderBy(F.desc("age")).rowsBetween(Window.currentRow+1,Window.unboundedFollowing)
subject_win_next = Window.partitionBy("subject_id").orderBy("age").rowsBetween(Window.currentRow+1,Window.unboundedFollowing)
#wi_next = Window.partitionBy("subject_id").orderBy(F.desc("age"))
for c in data.columns:
    if (c not in ("subject_id", "age", "gender", "hadm_id", "charlson_comorbidity_index")):
            data = data.withColumn('temp_' + c + 'prev', F.first(c, ignorenulls = True).over(subject_win_prev)) \
            .withColumn('temp_' + c + 'next', F.first(c, ignorenulls = True).over(subject_win_next)) \
            .withColumn(c , F.when(F.col(c).isNotNull() ,F.col(c)) \
                        .when(F.col(c).isNull() & F.col('temp_' + c + 'prev').isNull(), F.col('temp_' + c + 'next')) \
                        .when(F.col(c).isNull() & F.col('temp_' + c + 'next').isNull(), F.col('temp_' + c + 'prev')) \
                        .otherwise(((F.col('temp_' + c + 'prev') + F.col('temp_' + c + 'next'))/2))) \
            .drop('temp_' + c + 'prev', 'temp_' + c + 'next')

In [15]:
# same sample for verification
data.filter((data.subject_id == 10040025)).select("subject_id","age","basophils_abs", "db_wbc", "platelet", "ast").orderBy("age").show()

+-----------+-----+-------------+------+--------+----+
| subject_id|  age|basophils_abs|db_wbc|platelet| ast|
+-----------+-----+-------------+------+--------+----+
|1.0040025E7|64.21|         0.01|   8.9|   202.0|16.0|
|1.0040025E7|66.51|         0.01|   8.9|   238.5|16.0|
|1.0040025E7|66.56|         0.01|   8.9|   275.0|16.0|
|1.0040025E7| 66.8|         0.01|   6.2|   258.0|16.0|
|1.0040025E7|68.46|         0.01|  12.6|   281.0|16.0|
|1.0040025E7|68.59|         0.01|  11.3|   307.0|16.0|
|1.0040025E7|68.86|         0.06|   9.8|   275.0|16.0|
|1.0040025E7|68.93|         0.06|   8.6|   323.0|16.0|
|1.0040025E7|68.99|         0.06|  12.6|   366.0|16.0|
|1.0040025E7|69.06|         0.11|  16.7|   349.0|16.0|
+-----------+-----+-------------+------+--------+----+



In [16]:
data.toPandas().to_csv("../data/EDA/after_imputation_all.csv")

#### Remove columns with more than 50% missing values

In [17]:
dt = data.describe().toPandas()
dt

,summary,subject_id,dod,hadm_id,age,so2,po2,pco2,fio2,aado2,...,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,crp,charlson_comorbidity_index,gender
0,count,431231,0,431231,431231,96955,175431,175435,62205,23454,...,272881,54343,58222,191984,172593,21802,215023,79356,431231,431231
1,mean,1.5007664969607936E7,None,2.5003138264725868E7,59.26944906558195,90.49880666288485,182.50438633992852,46.79705588964574,64.11645687645687,490.24766010062245,...,1.1371222254389204,2.2347229633991574,1.3576929682937697,604.8320927785649,9.136013801254975,254.57928171727363,340.76348809197157,63.259554602046556,3.5870218050186558,None
2,stddev,2877497.806271312,None,2888180.402945383,19.207987144441066,13.240820356753373,129.76312570644527,13.354823688909741,24.818618914407583,123.24380218078677,...,2.8168865552570077,4.0850243618287365,1.837829309255241,7080.0965510149035,30.16613057602737,375.40763540589734,1125.383360538912,70.77766963419134,3.0318242331362244,None
3,min,1.0000032E7,None,2.0000019E7,18.0,7.0,0.0,10.0,21.0,0.12,...,0.1,0.1,0.1,4.0,1.0,3.0,31.0,0.1,0.0,F
4,max,1.9999987E7,None,2.9999928E7,103.17,100.0,4242.0,246.0,100.0,743.0,...,87.2,68.0,29.6,591950.0,673.0,7380.0,377000.0,608.1,20.0,M


In [18]:
# identify and get columns with < 50% missing values
dt_t = dt.T
print(dt_t.shape)
dt_t.columns = dt_t.iloc[0]
dt_t.drop(dt_t.index[0], inplace=True)
#dt_t.drop(dt_t.index[89], inplace=True)
dt_t["count"] = dt_t['count'].astype(int)
dt_t['missing_percentage'] = 100 - (dt_t["count"] / data.count())*100
print(dt_t)
dt_t = dt_t[dt_t["missing_percentage"] < 50]
print(dt_t)
print(dt_t.shape)

(90, 5)
summary                      count                  mean              stddev  \
subject_id                  431231  1.5007664969607936E7   2877497.806271312   
dod                              0                  None                None   
hadm_id                     431231  2.5003138264725868E7   2888180.402945383   
age                         431231     59.26944906558195  19.207987144441066   
so2                          96955     90.49880666288485  13.240820356753373   
...                            ...                   ...                 ...   
ggt                          21802    254.57928171727363  375.40763540589734   
ld_ldh                      215023    340.76348809197157   1125.383360538912   
crp                          79356    63.259554602046556   70.77766963419134   
charlson_comorbidity_index  431231    3.5870218050186558  3.0318242331362244   
gender                      431231                  None                None   

summary                        

After removing all the columns with missing value % >= 50, we get remaining 47 feature fields which are as follows.

In [19]:
print(dt_t.shape)
dt_t.index

(47, 6)


Index(['subject_id', 'hadm_id', 'age', 'bg2_lactate', 'db_wbc',
       'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',
       'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes',
       'monocytes', 'neutrophils', 'albumin', 'aniongap', 'bicarbonate', 'bun',
       'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium',
       'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv',
       'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est',
       'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total',
       'charlson_comorbidity_index', 'gender'],
      dtype='object')

In [20]:
# Filter out only the identified column data from the data
data = data.select(dt_t.index.values.tolist())

In [21]:
# round off age by 2 decimal point
print(data.printSchema())
data.describe().toPandas()

root
 |-- subject_id: double (nullable = true)
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bg2_lactate: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils: double (nullable = true)
 |-- lymphocytes: double (nullable = true)
 |-- monocytes: double (nullable = true)
 |-- neutrophils: double (nullable = true)
 |-- albumin: double (nullable = true)
 |-- aniongap: double (nullable = true)
 |-- bicarbonate: double (nullable = true)
 |-- bun: double (nullable = true)
 |-- calcium: double (nullable = true)
 |-- chloride: double (nullable = true)
 |-- creatinine: double (nullable = true)
 |-- glucose: double (nullable = true)
 |-- sodium: double (nullable = true)
 

,summary,subject_id,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,...,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender
0,count,431231,431231,431231,219710,387896,280296,280307,280412,280296,...,379279,431231,431231,430920,279523,273092,279081,272881,431231,431231
1,mean,1.5007664969607936E7,2.5003138264725868E7,59.26944906558195,2.297948614082201,10.699593808649754,0.05449114507519799,0.24053198814157833,2.160998281100673,0.7695476032479963,...,1.3990607969331283,0.1450684204057686,0.9563553176836034,1.2488995057087133,75.29124079234983,121.26987059306022,101.7731160487457,1.1371222254389208,3.5870218050186558,None
2,stddev,2877497.8062713156,2888180.402945385,19.207987144441052,1.9372557048716157,21.877885829231825,0.35698353870704,1.4242700752418933,8.55591268618894,3.1591814165100063,...,1.6458283468891914,0.35217021653027053,0.13624175008954154,1.4746186560465082,357.0620184326097,132.41562226334918,595.9465869121606,2.8168865552570073,3.0318242331362244,None
3,min,1.0000032E7,2.0000019E7,18.0,0.2,0.1,0.0,0.0,0.0,0.0,...,0.1,0.0,0.75,0.1,1.0,5.0,1.0,0.1,0.0,F
4,max,1.9999987E7,2.9999928E7,103.17,101.0,12500.0,125.0,625.0,2750.0,1375.0,...,80.0,1.0,1.32,80.0,61854.0,4128.0,42606.0,87.2,20.0,M


#### retain rows which has missing values less than 25% of the columns.

In [23]:
# calculate missing percentage for every row
from operator import add
from functools import reduce
for c in data.columns:
    if ('missing_' not in c) and (c not in ("subject_id", "age", "gender", "hadm_id", "charlson_comorbidity_index")):
            data = data.withColumn('missing_' + c, F.when(F.col(c).isNull(), 1).otherwise(0))

data = data.withColumn('missing_percentage', (reduce(add, [F.col(x) for x in data.columns if "missing_" in x])/(len(data.columns)-5))*100)
data.toPandas()

,subject_id,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,...,missing_wbc,missing_scr_min,missing_ckd,missing_mdrd_est,missing_scr_baseline,missing_alt,missing_alp,missing_ast,missing_bilirubin_total,missing_percentage
0,10001919.0,29897682.0,59.30,NaN,15.2,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,1,22.619048
1,10003385.0,23040642.0,41.73,NaN,13.7,0.06,0.17,2.990,0.95,9.46,...,0,0,0,0,0,0,0,0,0,3.571429
2,10006029.0,29107716.0,72.63,0.7,3.9,0.01,0.01,0.390,0.38,2.61,...,0,0,0,0,0,0,0,0,0,0.000000
3,10006029.0,27104518.0,72.74,1.4,6.4,0.01,0.01,0.390,0.38,2.61,...,0,0,0,0,0,0,0,0,0,0.000000
4,10006029.0,20850584.0,73.52,1.4,5.7,0.02,0.04,0.345,0.67,2.89,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19994379.0,21694878.0,68.33,0.9,9.6,0.05,0.16,1.940,0.85,6.60,...,0,0,0,0,0,0,0,0,0,0.000000
431227,19994379.0,27052619.0,71.34,1.8,11.6,0.06,0.10,0.750,0.73,5.63,...,0,0,0,0,0,0,0,0,0,0.000000
431228,19994379.0,27334101.0,71.41,6.9,12.9,0.04,0.28,1.180,1.24,7.01,...,0,0,0,0,0,0,0,0,0,0.000000
431229,19994379.0,23099193.0,71.50,3.3,7.6,0.04,0.23,0.830,0.69,4.39,...,0,0,0,0,0,0,0,0,0,0.000000


In [24]:
data.groupBy("missing_percentage").count().toPandas()

,missing_percentage,count
0,39.285714,284
1,0.000000,162731
2,21.428571,3181
3,14.285714,7720
4,15.476190,1669
5,9.523810,2368
6,11.904762,13123
7,41.666667,727
8,35.714286,404
9,40.476190,500


In [25]:
data3 = data.filter("missing_percentage < 25")

In [26]:
print(data.count())
print(data3.count())


431231
375194


In [27]:
431231 - 375194

56037

Removed 56037 rows with missing % > 25

In [28]:
print(len(data3.columns))
condition = lambda x: ("missing_" in x)
data3 = data3.drop(*filter(condition, data3.columns))
len(data3.columns)

90


47

In [29]:
data3.toPandas().to_csv("../data/EDA/after_eda_all.csv")

In [30]:
data4 = data3 #spark.read.option("header","true").csv("../data/EDA/after_eda_all.csv")
data4.printSchema()

root
 |-- subject_id: double (nullable = true)
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bg2_lactate: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils: double (nullable = true)
 |-- lymphocytes: double (nullable = true)
 |-- monocytes: double (nullable = true)
 |-- neutrophils: double (nullable = true)
 |-- albumin: double (nullable = true)
 |-- aniongap: double (nullable = true)
 |-- bicarbonate: double (nullable = true)
 |-- bun: double (nullable = true)
 |-- calcium: double (nullable = true)
 |-- chloride: double (nullable = true)
 |-- creatinine: double (nullable = true)
 |-- glucose: double (nullable = true)
 |-- sodium: double (nullable = true)
 

In [31]:
# get the first admission readings and last admission co-morbidity index value
from pyspark.sql.window import Window
import pyspark.sql.functions as F
subject_win = Window.partitionBy("subject_id").orderBy(("age"))
base_data = data4.withColumn("row",F.row_number().over(subject_win)) \
  .filter(F.col("row") == 1).drop("row", "charlson_comorbidity_index")

In [32]:
base_data.count()

138274

In [33]:
subject_win_predict = Window.partitionBy("subject_id").orderBy(F.desc("age"))
base_data_predict = data4.withColumn("row",F.row_number().over(subject_win_predict)) \
  .filter(F.col("row") == 1).select("subject_id", "charlson_comorbidity_index")
base_data_predict.count()

138274

In [34]:
print(base_data.columns)
print(base_data_predict.columns)

['subject_id', 'hadm_id', 'age', 'bg2_lactate', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'albumin', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total', 'gender']
['subject_id', 'charlson_comorbidity_index']


In [35]:
base_data.toPandas()

,subject_id,hadm_id,age,bg2_lactate,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,...,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,gender
0,10001919.0,29897682.0,59.30,NaN,15.2,NaN,NaN,NaN,NaN,NaN,...,15.2,1.0,0.0,1.07,1.00,NaN,NaN,NaN,NaN,M
1,10003385.0,23040642.0,41.73,NaN,13.7,0.06,0.17,2.99,0.95,9.46,...,13.7,1.1,0.0,1.14,1.10,16.0,81.0,13.0,0.4,M
2,10006029.0,29107716.0,72.63,0.7,3.9,0.01,0.01,0.39,0.38,2.61,...,3.9,1.3,1.0,1.03,1.30,123.0,241.0,58.0,2.3,M
3,10006431.0,25589898.0,66.03,1.2,10.2,0.03,0.20,0.97,1.07,2.86,...,10.2,0.7,0.0,0.81,0.70,70.0,306.0,57.0,8.8,F
4,10011231.0,29117639.0,45.50,NaN,4.4,0.03,0.14,1.30,0.18,2.75,...,4.4,0.6,0.0,0.87,0.60,NaN,NaN,NaN,NaN,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138269,19990486.0,25266731.0,89.92,NaN,8.6,NaN,NaN,NaN,NaN,NaN,...,8.6,2.7,1.0,0.77,2.70,NaN,NaN,NaN,NaN,F
138270,19990581.0,25446822.0,41.56,NaN,9.6,NaN,NaN,NaN,NaN,NaN,...,9.6,0.9,0.0,1.14,0.90,132.0,89.0,64.0,0.3,M
138271,19993951.0,23793933.0,72.13,2.3,10.6,0.03,0.12,1.44,0.59,5.03,...,10.6,1.7,0.0,1.04,1.04,21.0,115.0,31.0,1.5,M
138272,19994379.0,21694878.0,68.33,0.9,9.6,0.05,0.16,1.94,0.85,6.60,...,9.6,0.8,0.0,1.04,0.80,20.0,173.0,26.0,0.8,M


In [36]:
base_data_predict.toPandas()

,subject_id,charlson_comorbidity_index
0,10001919.0,7.0
1,10003385.0,0.0
2,10006029.0,13.0
3,10006431.0,8.0
4,10011231.0,0.0
...,...,...
138269,19990486.0,7.0
138270,19990581.0,3.0
138271,19993951.0,8.0
138272,19994379.0,7.0


In [37]:
base_data.toPandas().to_csv("../data/EDA/clustering_data_all.csv")
base_data_predict.toPandas().to_csv("../data/EDA/prediction_value_all.csv")